In [45]:
import pandas as pd;
import numpy as np;

In [71]:
url = "https://data.seattle.gov/resource/tazs-3rd5.json?Offense=Robbery"
df_roberry = pd.read_json(url)

df_roberry = df_roberry.drop(columns = [
    "_100_block_address", "beat", "mcpp", "offense_code"
    , "offense_end_datetime", "offense_id", "precinct"
    , "report_number", "sector", "report_datetime"
    , "group_a_b", "offense_parent_group", "crime_against_category"
])
#df_roberry.columns = ["lat", "long", "offense", "date"]
df_roberry.head()

,latitude,longitude,offense,offense_start_datetime
0,47.666384,-122.299552,Robbery,2020-02-02 20:30:00
1,47.605126,-122.330494,Robbery,2020-02-04 08:00:00
2,47.617605,-122.316239,Robbery,2020-02-04 00:00:00
3,47.602412,-122.331082,Robbery,2020-02-03 09:30:00
4,47.662129,-122.346712,Robbery,2020-01-30 18:30:00


In [73]:
url = "https://data.seattle.gov/resource/tazs-3rd5.json?Offense=Burglary"
df_burglary = pd.read_json(url)
df_burglary.head()

""


In [35]:
csvpath = "data/SPD_Crime_Data__2008-Present.csv"
df = pd.read_csv(csvpath, low_memory=False)
df.head()

,Report Number,Offense ID,Offense Start DateTime,Offense End DateTime,Report DateTime,Group A B,Crime Against Category,Offense Parent Group,Offense,Offense Code,Precinct,Sector,Beat,MCPP,100 Block Address,Longitude,Latitude
0,2020-044620,12605873663,2020-02-05 10:10:00,NaN,2020-02-05 11:24:31,A,SOCIETY,DRUG/NARCOTIC OFFENSES,Drug/Narcotic Violations,35A,W,Q,Q1,MAGNOLIA,32XX BLOCK OF 23RD AVE W,-122.385974,47.649387
1,2020-044452,12605598696,2020-02-03 08:00:00,2020-02-04 08:00:00,2020-02-05 10:06:28,A,PROPERTY,LARCENY-THEFT,Theft of Motor Vehicle Parts or Accessories,23G,N,J,J3,ROOSEVELT/RAVENNA,63XX BLOCK OF 5TH AVE NE,-122.323399,47.675118
2,2020-043929,12605576026,2020-02-04 18:00:00,2020-02-04 18:45:00,2020-02-05 09:53:08,A,PROPERTY,LARCENY-THEFT,Shoplifting,23C,W,K,K3,CHINATOWN/INTERNATIONAL DISTRICT,14XX BLOCK OF S LANE ST,-122.313437,47.596655
3,2020-043586,12605575743,2020-02-04 12:00:00,2020-02-04 13:00:00,2020-02-05 09:52:57,A,PROPERTY,LARCENY-THEFT,Shoplifting,23C,W,K,K3,CHINATOWN/INTERNATIONAL DISTRICT,14XX BLOCK OF S LANE ST,-122.313437,47.596655
4,2020-042362,12605575276,2020-02-03 13:00:00,2020-02-03 15:20:00,2020-02-05 09:52:39,A,PROPERTY,LARCENY-THEFT,Shoplifting,23C,W,K,K3,CHINATOWN/INTERNATIONAL DISTRICT,14XX BLOCK OF S LANE ST,-122.313437,47.596655


In [36]:
df.size

13941224

In [38]:
df.Offense.unique()

array(['Drug/Narcotic Violations',
       'Theft of Motor Vehicle Parts or Accessories', 'Shoplifting',
       'Robbery', 'Destruction/Damage/Vandalism of Property', 'Rape',
       'Kidnapping/Abduction', 'Driving Under the Influence',
       'Theft From Motor Vehicle', 'Motor Vehicle Theft', 'Wire Fraud',
       'Theft From Building', 'Counterfeiting/Forgery',
       'All Other Larceny', 'Pornography/Obscene Material',
       'Extortion/Blackmail', 'Pocket-picking', 'Arson', 'Sodomy',
       'Stolen Property Offenses',
       'False Pretenses/Swindle/Confidence Game',
       'Credit Card/Automated Teller Machine Fraud', 'Embezzlement',
       'Identity Theft', 'Weapon Law Violations', 'Impersonation',
       'Fondling', 'Peeping Tom', 'Hacking/Computer Invasion',
       'Drug Equipment Violations',
       'Human Trafficking, Commercial Sex Acts', 'Liquor Law Violations',
       'Bad Checks', 'Murder & Nonnegligent Manslaughter',
       'Sexual Assault With An Object', 'Purchasing Pros

In [ ]:
# Burglary/Breaking & Entering
# Robbery